In [ ]:
import math
import pandas as pd
import numpy as np

In [ ]:
#Calculating geoid heights and adding them as columns into their dataframes
tp = pd.read_table("testp.csv", sep=",")
tp["N"] = tp["h"] - tp["H"]

cp = pd.read_table("controlp.csv", sep=",")
cp["N"] = cp["h"] - cp["H"]

In [ ]:
#Calculation of A matrix based on control point coordinates
list0 = []
list1 = []
list2 = []
list3 = []

for i in range(len(cp)):
    c1 = np.cos(cp.at[i, "Lat"]*np.pi/180) * np.cos(cp.at[i, "Long"]*np.pi/180)
    c2 = np.cos(cp.at[i, "Lat"]*np.pi/180) * np.sin(cp.at[i, "Long"]*np.pi/180)
    c3 = np.sin(cp.at[i, "Lat"]*np.pi/180)
    list0.append(1)
    list1.append(c1)
    list2.append(c2)
    list3.append(c3)

array0 = np.array(list0)
array1 = np.array(list1)
array2 = np.array(list2)
array3 = np.array(list3)

A_matrix_c = np.column_stack((array0, array1, array2, array3)) #A matrix created from control point coordinates
A_matrix_T_c = A_matrix_c.transpose() #Transpose of A_matrix

l_matrix_c = np.array(cp["N"].to_numpy().reshape(-1,1)) #Reshaping l matrix to be used in calculations below

x_matrix = np.dot(np.linalg.inv(np.dot(A_matrix_T_c, A_matrix_c)), np.dot(A_matrix_T_c, l_matrix_c)) #Matrix of unknowns

v_matrix = np.subtract(np.dot(A_matrix_c, x_matrix), l_matrix_c) #Residual matrix

In [ ]:
#Error estimation of coefficients:
test1 = v_matrix * v_matrix

PVV_1 = math.sqrt(np.sum(test1))
m0_1 = math.sqrt(PVV_1 / (len(v_matrix)))

Qxx = np.linalg.inv(np.dot(A_matrix_T_c, A_matrix_c))

Qii = np.dot(np.dot(A_matrix_c, Qxx), A_matrix_T_c)

coef = []
for i in range(len(A_matrix_c)):
    coef.append(m0_1*math.sqrt(Qii[i,i]))

coef_final = np.array(coef).reshape(-1,1)

In [ ]:
#Calculations on test points
list4 = []
list5 = []
list6 = []
list7 = []

for i in range(len(tp)):
    c1 = np.cos(tp.at[i, "Lat"]*np.pi/180) * np.cos(tp.at[i, "Long"]*np.pi/180)
    c2 = np.cos(tp.at[i, "Lat"]*np.pi/180) * np.sin(tp.at[i, "Long"]*np.pi/180)
    c3 = np.sin(tp.at[i, "Lat"]*np.pi/180)
    list4.append(1)
    list5.append(c1)
    list6.append(c2)
    list7.append(c3)

array4 = np.array(list4)
array5 = np.array(list5)
array6 = np.array(list6)
array7 = np.array(list7)

l_matrix_t = np.array(tp["N"].to_numpy().reshape(-1,1))
A_matrix_t = np.column_stack((array4, array5, array6, array7))
v_matrix_t = np.subtract(np.dot(A_matrix_t, x_matrix), l_matrix_t)

tp["v"] = v_matrix_t #Adding geoid height differences of test points and the geoid model to the dataframe

In [ ]:
#Validation of the model using test points
test2 = v_matrix_t * v_matrix_t
PVV_2 = math.sqrt(np.sum(test2))
m0_2 = math.sqrt(PVV_2 / (len(v_matrix_t) - 1))

In [ ]:
print("m0 values:\nControl points:",round(m0_1*100,2),"cm\tTest points:",round(m0_2*100,2),"cm")

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import *

In [ ]:
#PLOTS
#Scatter plot of control and test points
bg = plt.imread("base2.tif")
extent = (28.74, 29.16, 40.89, 41.26)
plt.figure(figsize=(10, 10))
plt.scatter(cp["Long"], cp["Lat"], c=cp["N"], marker="o", cmap=plt.cm.inferno)
plt.scatter(tp["Long"], tp["Lat"], c=tp["N"], marker="^", cmap=plt.cm.inferno)
plt.colorbar(label="Geoid height [m]", fraction=0.040, pad=0.04)
plt.clim(vmin = 35.975, vmax = 36.825)
plt.imshow(bg, extent = extent)

In [ ]:
#3D view of the surface using control points
X = cp["Lat"]
Y = cp["Long"]
Z = cp["N"]

fig = plt.figure()
ax = Axes3D(fig)
ax.plot_trisurf(X, Y, Z, linewidth=1, antialiased=True, cmap = plt.cm.cool)
plt.show()

In [ ]:
import sys
!{sys.executable} -m pip install plotly
import plotly.graph_objects as go

In [ ]:
#Calculation and plotting of model surface
Z_model = np.dot(A_matrix_c, x_matrix)
cp["Z"] = np.array(Z_model).reshape(-1,1)

fig = go.Figure(data=
    go.Contour(
        z=cp["Z"],
        x=Y,
        y=X,
        contours=dict(
            coloring ='heatmap',
            showlabels = True,
            labelfont = dict( 
                size = 12,
                color = 'white',
            )
        )))
fig.update_yaxes(range=[40.89, 41.26])
fig.update_xaxes(range=[28.74, 29.16])
fig.update_layout(width=500, height=500)
fig.show()

In [ ]:
#Scatter plot of geoid height differences on test points:
bg = plt.imread("base.tif")
extent = (28.74, 29.16, 40.89, 41.26)
plt.figure(figsize=(10, 10))
plt.scatter(tp["Long"], tp["Lat"], c=tp["v"], marker="^", cmap=plt.cm.inferno)
plt.colorbar(label="Differences (cm)", fraction=0.040, pad=0.04)
plt.clim(vmin = -32.5, vmax = 22.5)
plt.imshow(bg, extent = extent)